In [8]:
import gensim
import pandas as pd
from gensim import corpora
from gensim import models
import pyLDAvis.gensim
from gensim.models import LdaModel
from gensim.corpora.dictionary import Dictionary
import warnings
warnings.filterwarnings("ignore")

In [5]:
df = pd.read_csv('database_sentiment_label.csv',parse_dates=['publishedAt','extractedAt'])
df.head()

,channelId,videoId,author,publishedAt,comment,extractedAt,comment_clean_lem,sentiment_label
0,UCK3inMNRNAVUleEbpDU1k2g,24Ob5JxhF4s,@SEBFRIT,2025-06-02 19:21:45+00:00,Merci à Odoo le sponsor cette vidéo ! Pour déc...,2025-07-01 11:46:10,"['odoo', 'sponsor', 'vidéo', 'découvrir', 'odo...",Positive
1,UCK3inMNRNAVUleEbpDU1k2g,24Ob5JxhF4s,@guishoo4700,2025-06-30 18:45:12+00:00,Fait une vidéo sur Tyler the creator stp,2025-07-01 11:46:10,"['vidéo', 'tyler', 'creator']",Neutral
2,UCK3inMNRNAVUleEbpDU1k2g,24Ob5JxhF4s,@Remi-B-Goode,2025-06-30 16:35:49+00:00,Lol le disclaimer😂,2025-07-01 11:46:10,['disclaimer'],Neutral
3,UCK3inMNRNAVUleEbpDU1k2g,24Ob5JxhF4s,@Guaph,2025-06-29 20:17:58+00:00,"où peut-on voir la série ""Earn and Alfred"" ???",2025-07-01 11:46:10,"['voir', 'série', 'earn', 'alfred']",Neutral
4,UCK3inMNRNAVUleEbpDU1k2g,24Ob5JxhF4s,@rockman156,2025-06-29 19:05:12+00:00,C'est pas un re upload ? Impression de déjà vu...,2025-07-01 11:46:10,"['upload', 'impression', 'critique', 'foireux'...",Positive


In [9]:
def corpus(df):
    corpus = [c for c in df.tolist() if len(c)!=0]
    print(f'Les 10 premiers documents du corpus sur les {len(corpus)} :\n {corpus[:5]}')
    unique_tokens = set(token for doc in corpus for token in doc)
    print(f'Nombre de tokens uniques dans le corpus: {len(unique_tokens)}, petit extrait des 10 premiers {list(unique_tokens)[:5]}')
    return corpus

In [10]:
# le dictionnaire
corpus = corpus(df['comment_clean_lem'])
dictionary = corpora.Dictionary(corpus)
# Bag-of-words representation of the documents.
corpus_vect = [dictionary.doc2bow(doc) for doc in corpus]

print('Number of unique tokens: %d' % len(dictionary))
print('Number of documents: %d' % len(corpus_vect))

# Set training parameters.
num_topics = 5
chunksize = 2000
passes = 20
iterations = 400
eval_every = None  # Don't evaluate model perplexity, takes too much time.

# Make an index to word dictionary.
temp = dictionary[0]  # This is only to "load" the dictionary.
id2word = dictionary.id2token

model = LdaModel(
    corpus=corpus_vect,
    id2word=id2word,
    chunksize=chunksize,
    alpha='auto',
    eta='auto',
    iterations=iterations,
    num_topics=num_topics,
    passes=passes,
    eval_every=eval_every
)

# 07 66 41 05 83  7H 9H 

Les 10 premiers documents du corpus sur les 1731 :
 ["['odoo', 'sponsor', 'vidéo', 'découvrir', 'odoo', 'clique', 'httpsw.odoo.comrcal', 'découverte', 'redécouvrir', 'aller', 'écouter', 'childish', 'important', 'bise']", "['vidéo', 'tyler', 'creator']", "['disclaimer']", "['voir', 'série', 'earn', 'alfred']", "['upload', 'impression', 'critique', 'foireux', 'impression']"]
Nombre de tokens uniques dans le corpus: 85, petit extrait des 10 premiers ['”', '6', '+', 'ö', '𝐝']


TypeError: doc2bow expects an array of unicode tokens on input, not a single string

In [ ]:
# Create a corpus from a list of texts
common_dictionary = Dictionary(common_texts)
common_corpus = [common_dictionary.doc2bow(text) for text in common_texts]

# Train the model on the corpus.
lda = LdaModel(common_corpus, num_topics=10)